# Advanced Features: Streaming & Function Calling

> **Created by [Build Fast with AI](https://www.buildfastwithai.com)**

This notebook explores advanced features of Gemini 3 Pro including streaming responses and function calling.

## What you'll learn:
- Streaming responses for better UX
- Function calling (tool use)
- Multi-turn function calling
- Structured output generation
- Advanced error handling

In [ ]:
!pip install -q google-generativeai

In [ ]:
import google.generativeai as genai
import os
import json
from IPython.display import Markdown, display
import time

In [ ]:
# Configure API key
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except:
    GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY', 'your-api-key-here')

genai.configure(api_key=GOOGLE_API_KEY)

## 1. Streaming Responses

Streaming allows you to display partial results as they're generated, improving user experience.

In [ ]:
model = genai.GenerativeModel('gemini-3-pro')

prompt = "Write a detailed explanation of how neural networks work."

print("Streaming response:\n")
response = model.generate_content(prompt, stream=True)

for chunk in response:
    print(chunk.text, end='', flush=True)

## 2. Streaming with Chat

In [ ]:
model = genai.GenerativeModel('gemini-3-pro')
chat = model.start_chat(history=[])

print("User: Explain quantum computing\n")
print("Assistant: ", end='')

response = chat.send_message("Explain quantum computing", stream=True)

for chunk in response:
    print(chunk.text, end='', flush=True)

## 3. Function Calling Basics

Function calling allows Gemini to interact with external tools and APIs.

In [ ]:
# Define a function that the model can call
def get_weather(location: str, unit: str = "celsius") -> dict:
    """Get the weather for a location.
    
    Args:
        location: The city and country, e.g., 'London, UK'
        unit: Temperature unit, 'celsius' or 'fahrenheit'
    
    Returns:
        Weather information dictionary
    """
    # Simulated weather data
    weather_data = {
        "location": location,
        "temperature": 22 if unit == "celsius" else 72,
        "unit": unit,
        "condition": "Partly cloudy",
        "humidity": 65,
        "wind_speed": 15
    }
    return weather_data

# Define the function schema for Gemini
weather_function = genai.protos.FunctionDeclaration(
    name="get_weather",
    description="Get the current weather for a location",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "location": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="The city and country, e.g., 'London, UK'"
            ),
            "unit": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="Temperature unit, 'celsius' or 'fahrenheit'",
                enum=["celsius", "fahrenheit"]
            )
        },
        required=["location"]
    )
)

In [ ]:
# Create a tool with the function
weather_tool = genai.protos.Tool(
    function_declarations=[weather_function]
)

# Initialize model with the tool
model = genai.GenerativeModel(
    'gemini-3-pro',
    tools=[weather_tool]
)

# Ask a question that requires the function
chat = model.start_chat()
response = chat.send_message("What's the weather like in Tokyo?")

# Check if model wants to call a function
if response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Model wants to call: {function_call.name}")
    print(f"With arguments: {dict(function_call.args)}")
    
    # Execute the function
    if function_call.name == "get_weather":
        result = get_weather(**dict(function_call.args))
        print(f"\nFunction result: {result}")
        
        # Send result back to model
        response = chat.send_message(
            genai.protos.Content(
                parts=[genai.protos.Part(
                    function_response=genai.protos.FunctionResponse(
                        name="get_weather",
                        response={"result": result}
                    )
                )]
            )
        )
        
        print("\nFinal response:")
        display(Markdown(response.text))

## 4. Multiple Function Declarations

In [ ]:
# Define multiple functions
def calculate_bmi(weight_kg: float, height_m: float) -> dict:
    """Calculate Body Mass Index."""
    bmi = weight_kg / (height_m ** 2)
    if bmi < 18.5:
        category = "Underweight"
    elif bmi < 25:
        category = "Normal weight"
    elif bmi < 30:
        category = "Overweight"
    else:
        category = "Obese"
    
    return {
        "bmi": round(bmi, 2),
        "category": category
    }

def convert_temperature(value: float, from_unit: str, to_unit: str) -> float:
    """Convert temperature between units."""
    if from_unit == "celsius" and to_unit == "fahrenheit":
        return (value * 9/5) + 32
    elif from_unit == "fahrenheit" and to_unit == "celsius":
        return (value - 32) * 5/9
    else:
        return value

# Define function schemas
bmi_function = genai.protos.FunctionDeclaration(
    name="calculate_bmi",
    description="Calculate Body Mass Index from weight and height",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "weight_kg": genai.protos.Schema(type=genai.protos.Type.NUMBER),
            "height_m": genai.protos.Schema(type=genai.protos.Type.NUMBER)
        },
        required=["weight_kg", "height_m"]
    )
)

temp_function = genai.protos.FunctionDeclaration(
    name="convert_temperature",
    description="Convert temperature between Celsius and Fahrenheit",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "value": genai.protos.Schema(type=genai.protos.Type.NUMBER),
            "from_unit": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                enum=["celsius", "fahrenheit"]
            ),
            "to_unit": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                enum=["celsius", "fahrenheit"]
            )
        },
        required=["value", "from_unit", "to_unit"]
    )
)

# Create tools
tools = genai.protos.Tool(
    function_declarations=[bmi_function, temp_function]
)

model = genai.GenerativeModel('gemini-3-pro', tools=[tools])
print("Model initialized with multiple functions!")

## 5. Automatic Function Execution

In [ ]:
def execute_function_call(function_call):
    """Execute a function call and return the result."""
    function_name = function_call.name
    function_args = dict(function_call.args)
    
    # Map function names to actual functions
    available_functions = {
        "get_weather": get_weather,
        "calculate_bmi": calculate_bmi,
        "convert_temperature": convert_temperature
    }
    
    if function_name in available_functions:
        return available_functions[function_name](**function_args)
    else:
        return {"error": f"Function {function_name} not found"}

# Test with automatic execution
chat = model.start_chat()
user_message = "Convert 25 degrees Celsius to Fahrenheit"
print(f"User: {user_message}\n")

response = chat.send_message(user_message)

# Handle function calls automatically
while response.candidates[0].content.parts[0].function_call:
    function_call = response.candidates[0].content.parts[0].function_call
    print(f"Calling function: {function_call.name}")
    print(f"Arguments: {dict(function_call.args)}\n")
    
    # Execute function
    result = execute_function_call(function_call)
    print(f"Result: {result}\n")
    
    # Send result back
    response = chat.send_message(
        genai.protos.Content(
            parts=[genai.protos.Part(
                function_response=genai.protos.FunctionResponse(
                    name=function_call.name,
                    response={"result": result}
                )
            )]
        )
    )

print("Assistant response:")
display(Markdown(response.text))

## 6. JSON Mode for Structured Output

In [ ]:
# Configure model for JSON output
model = genai.GenerativeModel(
    'gemini-3-pro',
    generation_config={
        "response_mime_type": "application/json"
    }
)

prompt = """
Generate a user profile with the following fields:
- name
- age
- occupation
- hobbies (list)
- address (object with street, city, country)

Create a profile for a software engineer.
"""

response = model.generate_content(prompt)
user_profile = json.loads(response.text)

print("Generated User Profile:")
print(json.dumps(user_profile, indent=2))

## 7. Combining Streaming with Function Calling

In [ ]:
model = genai.GenerativeModel(
    'gemini-3-pro',
    tools=[genai.protos.Tool(function_declarations=[weather_function])]
)

chat = model.start_chat()
print("User: What's the weather in Paris and should I bring an umbrella?\n")
print("Assistant: ", end='')

response = chat.send_message(
    "What's the weather in Paris and should I bring an umbrella?",
    stream=True
)

function_calls = []
for chunk in response:
    if chunk.candidates[0].content.parts[0].function_call:
        function_calls.append(chunk.candidates[0].content.parts[0].function_call)
    elif chunk.text:
        print(chunk.text, end='', flush=True)

# Handle any function calls
if function_calls:
    print("\n\n[Function call detected]")
    for fc in function_calls:
        result = execute_function_call(fc)
        print(f"Result: {result}")

## 8. Advanced: Building a Calculator Agent

In [ ]:
import math

# Define calculator functions
def calculate(operation: str, x: float, y: float = None) -> float:
    """Perform mathematical calculations."""
    operations = {
        "add": lambda a, b: a + b,
        "subtract": lambda a, b: a - b,
        "multiply": lambda a, b: a * b,
        "divide": lambda a, b: a / b if b != 0 else float('inf'),
        "power": lambda a, b: a ** b,
        "sqrt": lambda a, b: math.sqrt(a),
        "log": lambda a, b: math.log(a)
    }
    
    if operation in operations:
        return operations[operation](x, y) if y is not None else operations[operation](x, 0)
    return None

# Define the function schema
calc_function = genai.protos.FunctionDeclaration(
    name="calculate",
    description="Perform mathematical calculations including add, subtract, multiply, divide, power, sqrt, log",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "operation": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="The operation to perform",
                enum=["add", "subtract", "multiply", "divide", "power", "sqrt", "log"]
            ),
            "x": genai.protos.Schema(
                type=genai.protos.Type.NUMBER,
                description="First number"
            ),
            "y": genai.protos.Schema(
                type=genai.protos.Type.NUMBER,
                description="Second number (optional for unary operations)"
            )
        },
        required=["operation", "x"]
    )
)

calc_tool = genai.protos.Tool(function_declarations=[calc_function])
calc_model = genai.GenerativeModel('gemini-3-pro', tools=[calc_tool])

# Test the calculator
questions = [
    "What is 15 plus 27?",
    "What is the square root of 144?",
    "Calculate 2 to the power of 8"
]

for question in questions:
    print(f"\n{'='*60}")
    print(f"Q: {question}")
    
    chat = calc_model.start_chat()
    response = chat.send_message(question)
    
    if response.candidates[0].content.parts[0].function_call:
        fc = response.candidates[0].content.parts[0].function_call
        result = calculate(**dict(fc.args))
        
        response = chat.send_message(
            genai.protos.Content(
                parts=[genai.protos.Part(
                    function_response=genai.protos.FunctionResponse(
                        name="calculate",
                        response={"result": result}
                    )
                )]
            )
        )
    
    print(f"A: {response.text}")

## 9. Error Handling with Functions

In [ ]:
def safe_execute_function(function_call):
    """Safely execute a function call with error handling."""
    try:
        function_name = function_call.name
        function_args = dict(function_call.args)
        
        available_functions = {
            "calculate": calculate,
            "get_weather": get_weather,
        }
        
        if function_name not in available_functions:
            return {"error": f"Function {function_name} not available"}
        
        result = available_functions[function_name](**function_args)
        return {"success": True, "result": result}
        
    except TypeError as e:
        return {"error": f"Invalid arguments: {str(e)}"}
    except Exception as e:
        return {"error": f"Execution error: {str(e)}"}

# Test error handling
print("Testing safe function execution...\n")

# This will work
test_call = genai.protos.FunctionCall(
    name="calculate",
    args={"operation": "add", "x": 5, "y": 3}
)
print(f"Valid call result: {safe_execute_function(test_call)}")

## Next Steps

Now that you've mastered advanced features, explore:
- Building RAG systems with ChromaDB
- Creating autonomous agents with LangGraph
- Developing interactive Streamlit applications

---

## Learn More

Master advanced Generative AI techniques with the **[Gen AI Crash Course](https://www.buildfastwithai.com/genai-course)** by Build Fast with AI!

**Created by [Build Fast with AI](https://www.buildfastwithai.com)**